In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pdb
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras import backend as K
from matplotlib import pyplot as plt
%matplotlib inline
import scipy
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom
from scipy.ndimage import imread
import helpers
from models import DaveModel, DaveVGG, DaveVGG19, SimpleModel, LeNetModel
from trainer import Trainer

Using TensorFlow backend.


In [2]:
train = pd.read_json("_RawData/train.json/data/processed/train.json")
test = pd.read_json("_RawData/test.json/data/processed/test.json")

In [3]:
X = helpers.get_images(train)
X_test = helpers.get_images(test)

In [4]:
y = to_categorical(train.is_iceberg.values,num_classes=2)

In [5]:
Xtr, Xv, ytr, yv = train_test_split(X, y, shuffle=False, test_size=0.20)

In [6]:
trainRunner = Trainer([
    DaveModel(Xtr, ytr, Xv, yv),
    DaveVGG(Xtr, ytr, Xv, yv),
    DaveVGG19(Xtr, ytr, Xv, yv),
    LeNetModel(Xtr, ytr, Xv, yv)
])

In [7]:
trainRunner.models[0].model.load_weights("davemodel20171203-181713.h5")
trainRunner.models[1].model.load_weights("vgg20171203-182704.h5")
trainRunner.models[2].model.load_weights("vgg1920171203-183725.h5")
trainRunner.models[3].model.load_weights("lenet20171203-184024.h5")

In [18]:
results = pd.DataFrame()

for model in trainRunner.models:
    name = model.get_name()
    print(name)
    test = model.model.predict(X_test)
    results[name] = test[:, 0]

davemodel
vgg
vgg19
lenet


In [20]:
results.head(50)

,davemodel,vgg,vgg19,lenet
0,0.904778,0.506920,0.729676,0.523765
1,0.686788,0.277934,0.972092,0.611775
2,0.994779,0.248336,0.985353,0.991685
3,0.001994,0.011655,0.003505,0.004780
4,0.968177,0.280054,0.898017,0.604213
5,0.576131,0.680308,0.991889,0.034769
6,0.972794,0.978381,0.977407,0.918248
7,0.000298,0.004681,0.004760,0.000785
8,0.999843,1.000000,1.000000,0.999664
9,0.997944,1.000000,0.999995,0.999826
